# 요약문 작성 모델

In [71]:
from transformers import AutoTokenizer, BartForConditionalGeneration
import numpy as np
import itertools
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [136]:
model_name = "outputs/default-training/models/model-155epoch-102180steps-0.4105loss-0.9423acc"
# model_name = "alaggung/bart-r3f"
max_length = 200
num_beams = 5
length_penalty = 1.2

In [77]:
# 요약시킬 데이터
script=[
    '한나: 안드레아. 미팅에 관한 정보 받았어요?',
    '안드레아: 네, 받았어요. 고객서비스 책임자 핸슨씨가 미팅을 소집하고 날짜를 잡았어요.',
    '안드레아: 안 계신 동안 미팅 안건을 전달하셨어요.',
    '한나: 미팅이 언제죠?',
    '안드레아: 다음주 수요일이에요. 그리고 아침 11시에 시작해서 3시까지 진행되요. 핸슨씨가 회의를 진행하실 거에요.',
    '안드레아: 미팅에 참석하실 수 있으세요?',
    '한나: 음, 그럴 것 같아요. 근데 미팅은 무슨 내용이에요?',
    '안드레아: 문제 해결에 관한 미팅이에요.',
    '안드레아: 고객들의 고객 서비스에 대한 불만 사항들을 토론한데요.',
    '안드레아: 고객들이 우리의 서비스가 만족스럽지 않은 것이 판매감소요인이 되는 것 같으시데요.',
    '안드레아: 아마도 제 생각엔 고객들의 만족을 높이는 해답을 찾는 것일 것 같아요.',
    '안드레아: 핸슨씨가 저에게 직원들은 이 문제에 관한 생각들을 준비해오라고 하셨거든요.',
    '한나: 누가 미팅에 참석하시나요?',
    '안드레아: 모든 고객 서비스 직원들은 필수로 참석해야 한다고 들었어요. 인사부의 윌리암 프레슬리씨도 참석을 하신다고 했어요.',
    '한나: 오래 걸릴 것 같군요. 다과가 있을까요?',
    '안드레아: 방 뒤편에 다과를 준비해두실거에요'
]

In [65]:
script=[
    '과장: 우선 저희 오렌지 캬라멜 앨범 기획이 진행을,',
    '대리: 제가 준비한 것도 있고 지금 이따가 보여드리겠지만 좀 준비해 온 것도 있거든요.',
    '대리: 왜냐면 너희가 그냥 했는데 그냥 하는 건 아니잖아요.',
    '대리: 우리도 좀 준비한 게 있으니까 그것도 보면서 얘기했으면 좋겠고 고집을 피우고 싶은 건 아니에요.',
    '대리: 내가 고집을 피우고 싶은 건 아니고 대신 아까도 얘기했지만 새로운 걸 하고 싶은 거예요.',
    '대리: 참작의 고통을 같이 느끼고 싶다는 거지 그냥 있는 거 해서 우선 내가 준비한 것처럼 한 번 보면서 세상이나 이런 거는 예쁜 거 찾으면 예뻐 보이잖아 그런데 이걸 어떻게 꾸미냐도 되게 중요하거든요. 어떻게 보면',
    '대리: 뭐 남은 것도 아니고 그리고 이제 요가 자세 좀 재미난 자세들이 그러니까 이런 게 어떻게 보면 정말 뭐하는 게얘기했지 케이클러스 아기 친구 조정사장님이 지금 표정이 많이 안',
    '사원1: 급격히 어려워지셨습니다.',
    '사원2: 재밌는데 아무래도 저희는 이제 아이돌이다. 보니까 일단 지켜야 될 선은 그래도',
    '대리: 그런데 솔직히 여기 회사에서 체 변화 많이 하고 체 시도 많이 하고 어떻게 보면 정말 새로운 것들을 많이 하는 곳이 이곳인데 굳이 뭐 아이돌이어서 모여서',
    '사원2: 제 이건 개인적인 생각일 수 있는데 저는 오케이가 어느 정도는 보호는 해줘야 된다고 저는 생각을 그러니까 뭐 너무 가버리게 되면',
    '과장: 너무 웃긴 쪽으로',
    '사원2: 망가지는 거는 어느 정도는 그러니까 이게 이제 콘셉트 들어볼 때 정태호 님이 생각하시는 거는 어떻게 보면 되게 개그 쪽에 가까운 일단 저희가 비주얼 콘셉트에 있어서',
    '사원2: 저희 회사에서 잡은 걸로는 뮤직비디오 콘셉트은 저희가 생각한 거는 저희로서는 비주얼 콘셉트에서 이거는 이런 식으로 찍는다기보다는 이런 색감이나문양이라고 그러죠',
    '사원2: 예예 그런 것들을 좀 이용해보면 어떨까라는 생각을',
    '과장: 근데 사장님 저는 궁금한 게 이미 예를 들어서 회사에서 다 결정을 뭔가 콘셉트이나 다 내린 거라면은 여기 정태호 오빠도 하기 콘셉트 디렉터를',
    '대리: 누가 오류가 없어',
    '과장: 그게 무슨 의미가 되는 건지',
    '사원2: 그러니까 지금 너무 갭이 크면 안 될 것 같아요.',
    '대리: 그러니까 그거잖아요. 정해져 있는 거에 내가 색깔을 좀 입혀보자는 얘기잖아요.',
    '과장: 만약에 콘셉트 같은 게 정해져 있는 거를 정재호 오빠가 알았다면 그래서 다 준비하신 게 있는',
    '대리: 그러니까 매니저도 계속해서 진작에 얘기해서 그래야지 와서 계속 지금 입만 떠들고 입만 아프겠어요.',
    '사원1: 손을 들어서 넘어갈까요. 안무 영상 보는 걸로 넘어갈 거야',
    '대리: 아니 안무 누어 다 짰을 거 아니에요. 어차피',
    '대리: 기본적으로 짰는데 부장님도 마찬가지로 이게 지금 웃긴 거야 다 정해져 있는데 자꾸 와가지고 나 입만 아프고 무슨 나 시험을 나 싫어하는 것도 아니고아니 그 생각이 안 돼요',
    '대리: 굳이 정해져 있는 거면 얘기할 필요가 없으니까요. 아니 그러니까 뭐 통보를 해주든지 해야지 나도 오늘 지금 그냥 있는 것도 아니고 집에 있다고 나온 것도 아니고 인가 조건 촬영하고 있는데 그거 수탁 거르고 지금 나왔는데 아 나 손도 대었어',
    '과장: 얘기를 미리 해 주셨으면',
    '대리: 왜 이렇게 이렇게 오픈 소리 떨어지네 다 그럼 된 거 아니야 다 됐잖아 아무까지 다 나 했으면 정해줬으면 이거 제가 가져가도 되죠. 예 아니 바꿔 먹을까 진짜',
    '사원1: 정리해 줘요.',
    '과장: 그러면 그 의미가 없는데 그거를 미리 말씀을 해주니까 아니 그리 말해도 그렇게 우리 회사에서 알아서 다 정리하고 하고어느 정도 정해져 있는 거는 있어야 되게 만드는 것 같아요. 왜 이래요',
    '사원3: 아니 왜 이렇게들 안 속아 아니 나는 나는 나는 더 더 더',
    '대리: 제가 못하는 게 뭐냐 하면',
    '사원3: 내 편을 맞아 그 뭐라고 하니까 친구들이 내 편을 보여주지 나보고 누가 한 마디 말도 안 된다고 해야지 내가 이거 확 올라오는데',
    '대리: 근데 내가 일부러 자극시키려고 여기는 여기 안 해 그랬더니',
    '사원3: 또 따갑다가 아니 이거 원래 그렇게 하면 안 돼 너무 진짜 이렇게 착하게 하니까 잠깐만 그 이야기 화로 하고 냈어야 나 나 같을 때 뭔 말하는지 알아요.',
    '사원3: 사장님은 우리 도와주러 오셨는데 왜 저렇게 뛰게 말을 그리고 아빠가 usb 준비했는데 우리는 막 실컷 웃고 있는데 보고 막 정신 진짜 부사장님 왜 저러시나'
]

In [34]:
doc = '''우선 저희 오렌지 캬라멜 앨범 기획이 진행을
제가 준비한 것도 있고 지금 이따가 보여드리겠지만 좀 준비해 온 것도 있거든요. 
왜냐면 너희가 그냥 했는데 그냥 하는 건 아니잖아요. 
우리도 좀 준비한 게 있으니까 그것도 보면서 얘기했으면 좋겠고 고집을 피우고 싶은 건 아니에요. 
내가 고집을 피우고 싶은 건 아니고 대신 아까도 얘기했지만 새로운 걸 하고 싶은 거예요. 
참작의 고통을 같이 느끼고 싶다는 거지 그냥 있는 거 해서 우선 내가 준비한 것처럼 한 번 보면서 세상이나 이런 거는 예쁜 거 찾으면 예뻐 보이잖아 그런데 이걸 어떻게 꾸미냐도 되게 중요하거든요. 
어떻게 보면
뭐 남은 것도 아니고 그리고 이제 요가 자세 좀 재미난 자세들이 그러니까 이런 게 어떻게 보면 정말 뭐하는 게
얘기했지 케이클러스 아기 친구 조정
사장님이 지금 표정이 많이 안
급격히 어려워지셨습니다.
재밌는데 아무래도 저희는 이제 아이돌이다. 
보니까 일단 지켜야 될 선은 그래도
그런데 솔직히 여기 회사에서 체 변화 많이 하고 체 시도 많이 하고 어떻게 보면 정말 새로운 것들을 많이 하는 곳이 이곳인데 굳이 뭐 아이돌이어서 모여서
제 이건 개인적인 생각일 수 있는데 저는 오케이가 어느 정도는 보호는 해줘야 된다고 저는 생각을
그러니까 뭐 너무 가버리게 되면
너무 웃긴 쪽으로
망가지는 거는 어느 정도는 그러니까 이게 이제 콘셉트 들어볼 때 정태호 님이 생각하시는 거는 어떻게 보면 되게 개그 쪽에 가까운 일단 저희가 비주얼 콘셉트에 있어서 저희 회사에서 잡은 걸로는 뮤직비디오 콘셉트은 저희가 생각한 거는 저희로서는 비주얼 콘셉트에서 이거는 이런 식으로 찍는다기보다는 이런 색감이나
문양이라고 그러죠 예예 그런 것들을 좀 이용해보면 어떨까라는 생각을
근데 사장님 저는 궁금한 게 이미 예를 들어서 회사에서 다 결정을 뭔가 콘셉트이나 다 내린 거라면은 여기 정태호 오빠도 하기 콘셉트 디렉터를
누가 오류가 없어
그게 무슨 의미가 되는 건지
그러니까 지금 너무 갭이 크면 안 될 것 같아요.
그러니까 그거잖아요. 정해져 있는 거에 내가 색깔을 좀 입혀보자는 얘기잖아요.
만약에 콘셉트 같은 게 정해져 있는 거를 정재호 오빠가 알았다면 그래서 다 준비하신 게 있는
그러니까 매니저도 계속해서 진작에 얘기해서 그래야지 와서 계속 지금 입만 떠들고 입만 아프겠어요.
손을 들어서 넘어갈까요. 안무 영상 보는 걸로 넘어갈 거야
아니 안무 누어 다 짰을 거 아니에요. 
어차피
기본적으로 짰는데 부장님도 마찬가지로 이게 지금 웃긴 거야 다 정해져 있는데 자꾸 와가지고 나 입만 아프고 무슨 나 시험을 나 싫어하는 것도 아니고
아니 그 생각이 안 돼요 굳이 정해져 있는 거면 얘기할 필요가 없으니까요. 
아니 그러니까 뭐 통보를 해주든지 해야지 나도 오늘 지금 그냥 있는 것도 아니고 집에 있다고 나온 것도 아니고 인가 조건 촬영하고 있는데 그거 수탁 거르고 지금 나왔는데 아 나 손도 대었어
얘기를 미리 해 주셨으면
왜 이렇게 이렇게 오픈 소리 떨어지네 다 그럼 된 거 아니야 다 됐잖아 아무까지 다 나 했으면 정해줬으면 이거 제가 가져가도 되죠. 
예
아니 바꿔 먹을까 진짜
정리해 줘요.
그러면 그 의미가 없는데 그거를 미리 말씀을 해주니까 아니 그리 말해도 그렇게 우리 회사에서 알아서 다 정리하고 하고
어느 정도 정해져 있는 거는 있어야 되게 만드는 것 같아요.
왜 이래요
아니 왜 이렇게들 안 속아
아니 나는 나는 나는 더 더 더
제가 못하는 게 뭐냐 하면
내 편을 맞아 그 뭐라고 하니까 친구들이 내 편을 보여주지 나보고 누가 한 마디 말도 안 된다고 해야지 내가 이거 확 올라오는데
근데 내가 일부러 자극시키려고 여기는 여기 안 해 그랬더니
또 따갑다가 아니 이거 원래 그렇게 하면 안 돼 너무 진짜 이렇게 착하게 하니까 잠깐만 그 이야기 화로 하고 냈어야
나 나 같을 때 뭔 말하는지 알아요. 
사장님은 우리 도와주러 오셨는데 왜 저렇게 뛰게 말을 그리고 아빠가 usb 준비했는데 우리는 막 실컷 웃고 있는데 보고 막 정신 진짜 부사장님 왜 저러시나'''

In [133]:
# 요약 추론 모델
def inference(dialogue):
    tokenizer = AutoTokenizer.from_pretrained("resources/tokenizers/unigram_4K")
    model = BartForConditionalGeneration.from_pretrained(model_name)
    model.eval()
    print(model.eval())
    inputs = tokenizer("[BOS]" + "[SEP]".join(dialogue) + "[EOS]", return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        num_beams=num_beams,
        length_penalty=length_penalty,
        max_length=max_length,
        use_cache=True,
    )
    summarization = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summarization

In [140]:
model = BartForConditionalGeneration.from_pretrained(model_name)
model.config

BartConfig {
  "_name_or_path": "outputs/default-training/models/model-155epoch-102180steps-0.4105loss-0.9423acc",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 2,
  "classifier_dropout": 0.1,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 2,
  "decoder_start_token_id": 2,
  "dropout": 0.5,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "eos_token_id": 3,
  "forced_eos_token_id": 3,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "bart",
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "scale_embedding": false,
  "torch_dt

In [126]:
inference(script)

'아침과 설슨 중 미팅 있는 달'

## 키워드 분석 모델

In [80]:
doc = ' '.join(script)

In [93]:
mc = Mecab()

tokenized_doc = mc.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'NNG' and word[0] != '안드레아'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('한나', 'NR'), (':', 'SC'), ('안드레아', 'NNG'), ('.', 'SY'), ('미팅', 'NNG'), ('에', 'JKB'), ('관한', 'VV+ETM'), ('정보', 'NNG'), ('받', 'VV'), ('았', 'EP')]
명사 추출 : 미팅 정보 고객 서비스 책임자 미팅 소집 날짜 동안 미팅 안건 전달 미팅 다음 주 수요일 아침 시작 진행 회의 진행 미팅 참석 미팅 내용 문제 해결 미팅 고객 고객 서비스 불만 사항 토론 고객 서비스 만족 판매 감소 요인 생각 고객 만족 해답 직원 문제 생각 준비 미팅 참석 고객 서비스 직원 필수 참석 인사 부 참석 다과 방 뒤편 다과 준비


사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다.  
CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 쉽게 n-gram을 추출할 수 있기 때문입니다.  
예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과  
3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [94]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 113
trigram 다섯개만 출력 : ['감소 요인' '감소 요인 생각' '고객 고객' '고객 고객 서비스' '고객 만족']


## 키워드들을 수치화해주는 모델

In [95]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정한다.

In [96]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['회의 진행', '미팅 소집 날짜', '시작 진행 회의', '회의 진행 미팅', '미팅 소집']


# Max Sum Similarity

In [97]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [98]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['토론 고객 서비스', '토론 고객', '다음 수요일', '소집 날짜', '문제 해결 미팅']

# Maximal Marginal Relevance
결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다.  
MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다.  
참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드/키프레이즈를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다.  
먼저 문서와 가장 유사한 키워드/키프레이즈를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [99]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [100]:
keyword = mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)
keyword

['미팅 소집', '판매 감소 요인', '고객 만족 해답', '다음 수요일', '직원 문제']

### 특징 키워드가 포함된 문장들만 추출

In [101]:
# 자카드 유사도
def jaccard_similarity(keyword, text):
    key = set(keyword)
    txt = set(text)
    return float(len(key.intersection(txt)) / len(key.union(txt)))    

In [102]:
# 문장 형태소 분리
def get_pos(sentence):
    tokenized_doc = mc.pos(sentence)
    return [word[0] for word in tokenized_doc if word[1] == 'NNG']

In [127]:
for key in keyword:
    for txt in script:
        pos = get_pos(txt)
        sim = jaccard_similarity(key.split(), pos)
        if sim > 0:
            print('--원본 스크립트--')
            print(txt, f'유사도: {sim}')
            
            print('---요약본---')
            print(inference(txt))
            print('-' * 100)

--원본 스크립트--
한나: 안드레아. 미팅에 관한 정보 받았어요? 유사도: 0.25
---요약본---
선호하는 가장 많은 방문
----------------------------------------------------------------------------------------------------
--원본 스크립트--
안드레아: 네, 받았어요. 고객서비스 책임자 핸슨씨가 미팅을 소집하고 날짜를 잡았어요. 유사도: 0.2857142857142857
---요약본---
늦으면 미리 가기
----------------------------------------------------------------------------------------------------
--원본 스크립트--
안드레아: 안 계신 동안 미팅 안건을 전달하셨어요. 유사도: 0.16666666666666666
---요약본---
늦으면 미리 사유를 알려주기
----------------------------------------------------------------------------------------------------
--원본 스크립트--
한나: 미팅이 언제죠? 유사도: 0.5
---요약본---
지금 하는 것
----------------------------------------------------------------------------------------------------
--원본 스크립트--
안드레아: 미팅에 참석하실 수 있으세요? 유사도: 0.25
---요약본---
레석석과 보러 가기 위해 사용하는 것
----------------------------------------------------------------------------------------------------
--원본 스크립트--
한나: 음, 그럴 것 같아요. 근데 미팅은 무슨 내용이에요? 유사도: 0.3333333333333333
---요약본---
지금 하는 것이 선호하는지
---

In [123]:
for key in keyword:
    for txt in script:
        pos = get_pos(txt)
        print(key.split() in pos)
        if key.split() in pos.:
            print(txt)
#         pos = get_pos(txt)
#         sim = jaccard_similarity(key.split(), pos)
#         if sim > 0:
#             print('--원본 스크립트--')
#             print(txt, f'유사도: {sim}')
            
#             print('---요약본---')
#             print(inference(txt))
#             print('-' * 100)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
